# Excercise 3 : A day in the life

Description : นำข้อมูลกิจกรรมของแต่ละคนมาทำการวิเคราะห์ด้วย Python <br>
Date        : 26 February 2564 <br>
Change Logs : <br>
26 FEB 6 : เรียนรู้การใช้ USE,UMAP,DBSCAN <br>

In [5]:
# main library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os
import tensorflow as tf

%matplotlib inline

In [2]:
import tensorflow as tf
tf.__version__

'2.4.1'

# Import Data

In [7]:
import os
import glob
data_column = None
# get column name
for filepath in glob.glob('data/*.*'):
    name = os.path.splitext(filepath)[0]
    token_list = name.split('_')
    id_str = token_list[1]
    df_raw = pd.read_excel(filepath)
    data_column = df_raw.columns.to_list()
    break
data_column.append("ID")
job_df = pd.DataFrame(columns=data_column)

# get and append data
for filepath in glob.glob('data/*.*'):
    print(filepath)
    name = os.path.splitext(filepath)[0]
    token_list = name.split('_')
    id_str = token_list[1]
    excel_df = pd.read_excel(filepath)
    excel_df["ID"] = id_str
    job_df = job_df.append(excel_df,ignore_index=True)

data/Activities_6210422058.xlsx
data/Activities_6210422035.xlsx
data/Activities_6220422008.xlsx
data/Activities_6220422024.xlsx
data/Activities_6210422038.xlsx
data/Activities_6210422043.xlsx
data/Activities_6220422009.xlsx
data/Activities_6220422068.xls
data/activities_6220422013.xlsx
data/Activities_6220422005.xlsx
data/Activities_6310422091.xls
data/Activities_6310422085.xls
data/activities_6220422026.xls
data/activities_6210422044.xlsx
data/activities_6220422027.xls
data/activities_6220422002.xlsx
data/Activities_6220422106.xlsx
data/activities_6220422025.xls
data/Activities_6220422031.xls
data/Activities_6220422014.xlsx
data/Activities_6210422048.xlsx
data/Activities_6220422020.xls
data/Activities_6210422065.xlsx
data/Activities_6210422049.xlsx
data/Activities_6220422107.xlsx
data/Activities_6310422096.xls
data/Activities_6310422088.xlsx
data/activities_6210422034.xls
data/Activities_6210422007.xlsx
data/Activities_6220422004.xls
data/Activities_6220422010.xls
data/Activities_6210

In [2]:
job_df = pd.read_csv('adayinlife.csv')

In [3]:
job_df.head()

,time,activity,pain,gain,stu_id,time_type,time_hr,time_min
0,08.30,ตื่นนอน,ไม่อยากตื่น อยากนอนต่อ,ได้เริ่มต้นวันใหม่,6210422033,NaN,08,30
1,08.35,อาบน้ำ,ยังอยากนอนต่อ,สดชื่น+สะอาด,6210422033,NaN,08,35
2,09.00,กินอาหาเช้า,ต้องสั่ง Delivery,อิ่มท้อง,6210422033,NaN,09,0
3,10.00,เดินทางด้วย BTS ไปทำงาน,ราคาสูง,เดินทางสะดวกสบาย,6210422033,NaN,10,0
4,10.30,ถึงที่ทำงาน เริ่มทำงาน,NaN,เป็นช่วงที่พร้อมทำงานที่สุด,6210422033,NaN,10,30


# Grouping time data
reduce time data into 30 minute range group

In [4]:
job_df.loc[job_df['time_min'] <30, 'time_min_adj'] = '00'
job_df.loc[job_df['time_min'] >=30, 'time_min_adj'] = '30'
job_df.head()

,time,activity,pain,gain,stu_id,time_type,time_hr,time_min,time_min_adj
0,08.30,ตื่นนอน,ไม่อยากตื่น อยากนอนต่อ,ได้เริ่มต้นวันใหม่,6210422033,NaN,08,30,30
1,08.35,อาบน้ำ,ยังอยากนอนต่อ,สดชื่น+สะอาด,6210422033,NaN,08,35,30
2,09.00,กินอาหาเช้า,ต้องสั่ง Delivery,อิ่มท้อง,6210422033,NaN,09,0,00
3,10.00,เดินทางด้วย BTS ไปทำงาน,ราคาสูง,เดินทางสะดวกสบาย,6210422033,NaN,10,0,00
4,10.30,ถึงที่ทำงาน เริ่มทำงาน,NaN,เป็นช่วงที่พร้อมทำงานที่สุด,6210422033,NaN,10,30,30


In [5]:
job_df['time_adj'] = job_df.time_hr +':'+ job_df.time_min_adj
job_df.head()

,time,activity,pain,gain,stu_id,time_type,time_hr,time_min,time_min_adj,time_adj
0,08.30,ตื่นนอน,ไม่อยากตื่น อยากนอนต่อ,ได้เริ่มต้นวันใหม่,6210422033,NaN,08,30,30,08:30
1,08.35,อาบน้ำ,ยังอยากนอนต่อ,สดชื่น+สะอาด,6210422033,NaN,08,35,30,08:30
2,09.00,กินอาหาเช้า,ต้องสั่ง Delivery,อิ่มท้อง,6210422033,NaN,09,0,00,09:00
3,10.00,เดินทางด้วย BTS ไปทำงาน,ราคาสูง,เดินทางสะดวกสบาย,6210422033,NaN,10,0,00,10:00
4,10.30,ถึงที่ทำงาน เริ่มทำงาน,NaN,เป็นช่วงที่พร้อมทำงานที่สุด,6210422033,NaN,10,30,30,10:30


In [6]:
# จำนวนผู้คนที่นำมาวิเคราห์
print(f'จำนวนคนที่นำมาวิเคราะห์ = {job_df.stu_id.nunique()}')

จำนวนคนที่นำมาวิเคราะห์ = 46


# Universal Sentence Encoder (USE)
เป็น Multilingual Unversal Sentence Encoder ที่ช่วยในการทำ sentence embedding และบอกความเหมือนหรือคล้ายกันของประโยคได้โดยประโยคที่คล้ายๆ กันจะได้ผลลัพธ์ที่อยู่ใกล้ๆ กัน และรองรับหลายภาษา ทำให้สามารถใช้บอกความเหมือนของประโยคข้ามภาษาได้

In [7]:
import tensorflow_hub as hub
import tensorflow_text
import json
import umap

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
  return model(input)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


### a) Analyst pain data

In [26]:
df_pain = job_df['pain'].astype(str)
# df_pain.drop(df_pain[df_pain.pain=='nan'].index, inplace=True)

print(type(df_pain))
df_pain.dropna(inplace= True)
df_pain.head()
# df_gain = job_df['gain'].astype(str)
# df_activity = job_df['activity'].astype(str)
df_pain_new = pd.DataFrame(embed(df_pain.values).numpy(),index=df_pain.values)
df_pain_new.head(100)

<class 'pandas.core.series.Series'>


,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
ไม่อยากตื่น อยากนอนต่อ,0.054364,0.011224,0.040745,-0.010882,0.030455,-0.006305,-0.045503,-0.057252,-0.044143,-0.000255,...,0.055189,-0.127345,0.075504,0.020393,-0.001009,0.087807,-0.007657,0.057417,0.062247,-0.114787
ยังอยากนอนต่อ,0.021196,-0.018706,0.053195,0.033285,-0.042164,0.007284,-0.016017,-0.044495,-0.016801,0.072139,...,0.017318,-0.146370,0.049791,0.074899,-0.029245,0.032482,-0.065387,-0.068484,0.041623,-0.069284
ต้องสั่ง Delivery,0.007455,-0.027119,0.040350,0.024139,0.044342,0.050177,0.032860,0.011546,0.060301,-0.077157,...,0.001429,-0.109278,0.003805,0.025230,0.005108,-0.030433,0.053640,-0.064422,0.028076,-0.071393
ราคาสูง,0.052045,0.047519,0.049864,0.083479,-0.036867,0.097645,-0.009137,-0.021924,0.063269,0.014274,...,-0.013998,-0.153052,0.044795,-0.003492,-0.063761,-0.022144,-0.011883,-0.022044,0.051355,-0.045201
nan,0.024017,-0.069871,0.003002,0.015576,0.000886,0.013951,-0.052353,-0.058664,-0.008204,0.046010,...,-0.028139,-0.090141,-0.003964,0.024919,-0.007066,-0.026090,0.007892,-0.036901,-0.029454,-0.065358
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
nan,0.024017,-0.069871,0.003002,0.015576,0.000886,0.013951,-0.052353,-0.058664,-0.008204,0.046010,...,-0.028139,-0.090141,-0.003964,0.024919,-0.007066,-0.026090,0.007892,-0.036901,-0.029454,-0.065358
ถ้าไม่กินรู้สึกง่วงนอน เพราะติดแล้ว,-0.011888,0.054036,0.033016,0.037462,0.032030,-0.025838,-0.053623,-0.049325,0.038967,0.006195,...,0.024600,-0.135538,0.038349,0.035251,0.033096,-0.021557,-0.003085,0.010791,0.040169,-0.062790
nan,0.024017,-0.069871,0.003002,0.015576,0.000886,0.013951,-0.052353,-0.058664,-0.008204,0.046010,...,-0.028139,-0.090141,-0.003964,0.024919,-0.007066,-0.026090,0.007892,-0.036901,-0.029454,-0.065358
nan,0.024017,-0.069871,0.003002,0.015576,0.000886,0.013951,-0.052353,-0.058664,-0.008204,0.046010,...,-0.028139,-0.090141,-0.003964,0.024919,-0.007066,-0.026090,0.007892,-0.036901,-0.029454,-0.065358


# Uniform Manifold Approximation and Projection (UMAP)
ช่วยในการลดมิติของข้อมูลลงเช่นเดียวกับ t-SNE แต่ทำงานได้ดีกว่า t-SNE

In [33]:
pain_umap = umap.UMAP(random_state=999, n_neighbors=5).fit_transform(df_pain_new)

In [45]:
print(pain_umap)

[[ 4.8154507  9.113254 ]
 [ 5.1172647 11.430368 ]
 [ 5.711913   8.404788 ]
 ...
 [ 4.4641814 10.405332 ]
 [ 5.70778    7.2336206]
 [-8.385683  11.922481 ]]


# DBScan
จัดกลุ่มข้อมูด้วย DBScan

In [28]:
from sklearn.cluster import DBSCAN

In [34]:
label_pain = DBSCAN(eps=1,min_samples=3).fit_predict(pain_umap)

In [36]:
df_result_pain = pd.DataFrame({'pain':df_pain,'cluster':label_pain})
print(df_result_pain.cluster.nunique())
df_result_pain.head(100)

14


,pain,cluster
0,ไม่อยากตื่น อยากนอนต่อ,0
1,ยังอยากนอนต่อ,0
2,ต้องสั่ง Delivery,0
3,ราคาสูง,0
4,nan,1
...,...,...
95,nan,1
96,ถ้าไม่กินรู้สึกง่วงนอน เพราะติดแล้ว,0
97,nan,1
98,nan,1


In [43]:
cluster_list = df_result_pain['cluster'].unique()
cluster_list.sort()
for cluster in cluster_list:
    temp = df_result_pain[df_result_pain['cluster'] == cluster]
    display(temp)

,pain,cluster
0,ไม่อยากตื่น อยากนอนต่อ,0
1,ยังอยากนอนต่อ,0
2,ต้องสั่ง Delivery,0
3,ราคาสูง,0
7,อยากพักต่อ,0
...,...,...
910,เสียเวลาในการเตรียมอาหาร,0
911,อาหารเหลือ ล้างจานเยอะ,0
913,หนาว ทำให้ผิวแห้ง,0
914,มือถือแบตหมด,0


,pain,cluster
4,nan,1
6,nan,1
8,nan,1
9,nan,1
14,nan,1
...,...,...
893,nan,1
896,nan,1
899,nan,1
908,nan,1


,pain,cluster
5,มีร้านให้เลือกน้อย,2
23,หิว,2
26,หิว,2
30,หิว,2
77,หิว,2
322,หิว,2
575,หิว,2
577,หิว,2
580,หิว,2
596,หิว,2


,pain,cluster
18,ง่วงนอน,3
24,ง่วงนอน,3
111,ง่วงนอน แต่ต้องทำการบ้าน,3
172,ง่วงนอน,3
173,ง่วงนอน,3
280,กินตรงโต๊ะทำงานจะทำให้ส่งกลิ่น,3
309,ง่วงนอน นอนไม่เต็มอิ่ม,3
319,ง่วงนอน,3
320,ง่วงนอน,3
394,ง่วงนอน,3


,pain,cluster
39,หนาว,4
49,หนาว,4
154,หนาว,4
223,หนาว,4
237,หนาว,4
259,หนาว,4
274,หนาว,4
557,หนาว,4
569,หนาว,4
604,หนาว ผิวแห้ง,4


...

,pain,cluster
138,งานยังไม่เสร็จ,9
140,งานยังไม่เสร็จ,9
142,งานยังไม่เสร็จ,9
145,งานยังไม่เสร็จ,9
284,งานยังไม่เสร็จ,9
288,งานยังไม่เสร็จ,9
290,งานยังไม่เสร็จ,9
805,งานยังไม่เสร็จ,9


,pain,cluster
157,ขี้เกียจ,10
167,หนาว ขี้เกียจ,10
275,ขี้เกียจ,10
428,ง่วงนอน ขี้เกียจ,10
435,ขี้เกียจ,10
778,ขี้เกียจ,10
810,เหนื่อย ขี้เกียจ,10
832,เริ่มง่วงนอน ขี้เกียจ,10


,pain,cluster
161,ไม่รู้จะกินอะไรดี,11
208,ไม่รู้จะกินอะไรดี,11
310,คิดไม่ออก ไม่รู้จะกินอะไรดี,11
821,ไม่รู้จะกินอะไรดี,11


,pain,cluster
306,เหนื่อย เมื่อย,12
417,เหนื่อย และ ร้อน,12
610,เหนื่อย เบื่อ,12
612,เหนื่อย เบื่อ,12
614,เหนื่อย เบื่อ,12
621,เหนื่อย ท้อแท้,12
808,เหนื่อย ยาก,12
912,เหนื่อย ใช้แรงเยอะ,12


,pain,cluster
408,ขี้เกียจทำ น่าเบื่อ,13
410,น่าเบื่อ เมนูซ้ำซาก,13
413,ง่วงนอน น่าเบื่อ,13
437,น่าเบื่อ,13
439,น่าเบื่อ,13
859,น่าเบื่อ,13


In [10]:
# add thai font
# if need in colab => https://www.facebook.com/groups/colab.thailand/permalink/1421960354645985/
import matplotlib as mpl
plt.rcParams['font.family']='TH Sarabun New'
mpl.font_manager.fontManager.addfont('./fonts/THSarabunNew.ttf')
mpl.rc('font', family='TH Sarabun New', size=10)